In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import altair as alt
import ipywidgets as widgets

from datetime import date
from ipywidgets import interactive, interact

pd.set_option('display.max_columns', 999)
alt.data_transformers.disable_max_rows() #línea cambiada según instrucciones
alt.themes.enable('opaque')


%matplotlib inline

In [2]:
start_date = pd.to_datetime("2020-04-13")

In [3]:
# https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto6
confirmados = (
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto6/bulk/data.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
    .assign(fecha=lambda x: pd.to_datetime(x["fecha"]))
    .loc[lambda x: x["fecha"] >= start_date]
    .dropna()
    .astype({"casos_confirmados": np.float, "tasa": np.float})
)

confirmados.head()

,poblacion,casos_confirmados,fecha,region_id,region,provincia_id,provincia,comuna_id,comuna,tasa
0,247552.0,10630.0,2020-12-04,15.0,Arica y Parinacota,151.0,Arica,15101.0,Arica,4294.047311
1,247552.0,10443.0,2020-11-20,15.0,Arica y Parinacota,151.0,Arica,15101.0,Arica,4218.507627
3,247552.0,6783.0,2020-08-28,15.0,Arica y Parinacota,151.0,Arica,15101.0,Arica,2740.030377
4,247552.0,10586.0,2020-11-30,15.0,Arica y Parinacota,151.0,Arica,15101.0,Arica,4276.273268
5,247552.0,3372.0,2020-07-20,15.0,Arica y Parinacota,151.0,Arica,15101.0,Arica,1362.138056


In [4]:
activos = (
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto19/CasosActivosPorComuna.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
    .loc[lambda x: x["codigo_comuna"].notnull()]
    .melt(id_vars=["region", "codigo_region", "comuna", "codigo_comuna", "poblacion"], var_name="fecha", value_name="casos_activos")
    .assign(fecha=lambda x: pd.to_datetime(x["fecha"]))
    .loc[lambda x: x["fecha"] >= start_date]
)

activos.head()

,region,codigo_region,comuna,codigo_comuna,poblacion,fecha,casos_activos
0,Arica y Parinacota,15,Arica,15101.0,247552.0,2020-04-13,88.0
1,Arica y Parinacota,15,Camarones,15102.0,1233.0,2020-04-13,0.0
2,Arica y Parinacota,15,General Lagos,15202.0,810.0,2020-04-13,0.0
3,Arica y Parinacota,15,Putre,15201.0,2515.0,2020-04-13,0.0
4,Tarapaca,1,Alto Hospicio,1107.0,129999.0,2020-04-13,8.0


In [5]:
fallecidos = (
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto14/FallecidosCumulativo.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
    .melt(id_vars=["region"], var_name="fecha", value_name="fallecidos")
    .assign(
        fecha=lambda x: pd.to_datetime(x["fecha"]),
    )
    .loc[lambda x: x["fecha"] >= start_date]
)

fallecidos.head()

,region,fecha,fallecidos
374,Arica y Parinacota,2020-04-13,1.0
375,Tarapacá,2020-04-13,0.0
376,Antofagasta,2020-04-13,1.0
377,Atacama,2020-04-13,0.0
378,Coquimbo,2020-04-13,0.0


In [6]:
# https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto10
fallecidos_etareo = (
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto10/FallecidosEtario.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
    .melt(id_vars=["grupo_de_edad"], var_name="fecha", value_name="fallecidos")
    .assign(
        fecha=lambda x: pd.to_datetime(x["fecha"]),
        grupo_de_edad=lambda x: x["grupo_de_edad"].str.replace("<=39", "0-39")
    )
    .loc[lambda x: x["fecha"] >= start_date]
)

fallecidos_etareo.head()

,grupo_de_edad,fecha,fallecidos
28,0-39,2020-04-13,3
29,40-49,2020-04-13,2
30,50-59,2020-04-13,3
31,60-69,2020-04-13,12
32,70-79,2020-04-13,29


In [7]:
fallecidos["region"].unique()

array(['Arica y Parinacota', 'Tarapacá', 'Antofagasta', 'Atacama',
       'Coquimbo', 'Valparaíso', 'Metropolitana', 'O’Higgins', 'Maule',
       'Ñuble', 'Biobío', 'Araucanía', 'Los Ríos', 'Los Lagos', 'Aysén',
       'Magallanes', 'Total'], dtype=object)

In [8]:
activos["region"].unique()

array(['Arica y Parinacota', 'Tarapaca', 'Antofagasta', 'Atacama',
       'Coquimbo', 'Valparaiso', 'Metropolitana',
       'Del Libertador General Bernardo O’Higgins', 'Maule', 'Nuble',
       'Biobio', 'La Araucania', 'Los Rios', 'Los Lagos', 'Aysen',
       'Magallanes y la Antartica'], dtype=object)

In [9]:
confirmados["region"].unique()

array(['Arica y Parinacota', 'Tarapacá', 'Antofagasta', 'Atacama',
       'Coquimbo', 'Valparaíso', 'Metropolitana de Santiago',
       "Libertador General Bernardo O'Higgins", 'Maule', 'Ñuble',
       'Biobío', 'La Araucanía', 'Los Ríos', 'Los Lagos',
       'Aysén del General Carlos Ibáñez del Campo',
       'Magallanes y de la Antártica Chilena'], dtype=object)

In [10]:
region_names = {
    "Araucanía": "La Araucanía",
    "Tarapaca":"Tarapacá",
    "Valparaiso":"Valparaíso",
    "Los Rios":"Los Ríos",
    "Aysen": "Aysén del General Carlos Ibáñez del Campo",
    "Magallanes y la Antartica": "Magallanes y de la Antártica Chilena",
    "Metropolitana": "Metropolitana de Santiago",
    "O’Higgins": "Libertador General Bernardo O'Higgins",
}
activos=activos.replace({"region": region_names})
activos["region"].unique()


# fallecidos_array(['Arica y Parinacota', 'Tarapacá', 'Antofagasta', 'Atacama',
#        'Coquimbo', 'Valparaíso', 'Metropolitana de Santiago',
#        "Libertador General Bernardo O'Higgins", 'Maule', 'Ñuble',
#        'Biobío', 'La Araucanía', 'Los Ríos', 'Los Lagos',
#        'Aysén del General Carlos Ibáñez del Campo',
#        'Magallanes y de la Antártica Chilena', 'Total'], dtype=object


array(['Arica y Parinacota', 'Tarapacá', 'Antofagasta', 'Atacama',
       'Coquimbo', 'Valparaíso', 'Metropolitana de Santiago',
       'Del Libertador General Bernardo O’Higgins', 'Maule', 'Nuble',
       'Biobio', 'La Araucania', 'Los Ríos', 'Los Lagos',
       'Aysén del General Carlos Ibáñez del Campo',
       'Magallanes y de la Antártica Chilena'], dtype=object)

In [11]:
confirmados_rename=confirmados.rename(columns={"comuna_id":"codigo_comuna"})
merge1=activos.merge(confirmados_rename, how="inner", on=["codigo_comuna","fecha","region", "poblacion", "comuna"])
#Se hace el merge en todas las columnas coincidentes pese a que se puede perder información para no sobrepasar la memoria.

# merge1=merge1.T.drop_duplicates().T   Este comando no se me ejecutó, pero ya que el objetivo es solo graficar, no se elimanarán las columnas 
merge1                                 #duplicadas pese a que sean indeseables, puesto que no obstaculizan la ejecución del problema

,region,codigo_region,comuna,codigo_comuna,poblacion,fecha,casos_activos,casos_confirmados,region_id,provincia_id,provincia,tasa
0,Arica y Parinacota,15,Arica,15101.0,247552.0,2020-04-13,88.0,115.0,15.0,151.0,Arica,46.454886
1,Arica y Parinacota,15,Camarones,15102.0,1233.0,2020-04-13,0.0,0.0,15.0,151.0,Arica,0.000000
2,Arica y Parinacota,15,General Lagos,15202.0,810.0,2020-04-13,0.0,0.0,15.0,152.0,Parinacota,0.000000
3,Arica y Parinacota,15,Putre,15201.0,2515.0,2020-04-13,0.0,0.0,15.0,152.0,Parinacota,0.000000
4,Tarapacá,1,Alto Hospicio,1107.0,129999.0,2020-04-13,8.0,14.0,1.0,11.0,Iquique,10.769314
...,...,...,...,...,...,...,...,...,...,...,...,...
11691,Magallanes y de la Antártica Chilena,12,Primavera,12302.0,694.0,2020-12-04,5.0,24.0,12.0,123.0,Tierra del Fuego,3458.213256
11692,Magallanes y de la Antártica Chilena,12,Punta Arenas,12101.0,141984.0,2020-12-04,398.0,13141.0,12.0,121.0,Magallanes,9255.268199
11693,Magallanes y de la Antártica Chilena,12,San Gregorio,12104.0,681.0,2020-12-04,0.0,29.0,12.0,121.0,Magallanes,4258.443465
11694,Magallanes y de la Antártica Chilena,12,Timaukel,12303.0,282.0,2020-12-04,0.0,20.0,12.0,123.0,Tierra del Fuego,7092.198582


In [12]:
merge2=merge1.merge(fallecidos, how="inner", on=["fecha","region"])
#Se hace el merge en todas las columnas coincidentes pese a que se puede perder información para no sobrepasar la memoria.

merge2

,region,codigo_region,comuna,codigo_comuna,poblacion,fecha,casos_activos,casos_confirmados,region_id,provincia_id,provincia,tasa,fallecidos
0,Arica y Parinacota,15,Arica,15101.0,247552.0,2020-04-13,88.0,115.0,15.0,151.0,Arica,46.454886,1.0
1,Arica y Parinacota,15,Camarones,15102.0,1233.0,2020-04-13,0.0,0.0,15.0,151.0,Arica,0.000000,1.0
2,Arica y Parinacota,15,General Lagos,15202.0,810.0,2020-04-13,0.0,0.0,15.0,152.0,Parinacota,0.000000,1.0
3,Arica y Parinacota,15,Putre,15201.0,2515.0,2020-04-13,0.0,0.0,15.0,152.0,Parinacota,0.000000,1.0
4,Tarapacá,1,Alto Hospicio,1107.0,129999.0,2020-04-13,8.0,14.0,1.0,11.0,Iquique,10.769314,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7951,Los Lagos,10,Puyehue,10304.0,11787.0,2020-12-04,1.0,215.0,10.0,103.0,Osorno,1824.043438,213.0
7952,Los Lagos,10,Quemchi,10209.0,8783.0,2020-12-04,14.0,176.0,10.0,102.0,Chiloé,2003.871115,213.0
7953,Los Lagos,10,Quinchao,10210.0,8298.0,2020-12-04,24.0,94.0,10.0,102.0,Chiloé,1132.803085,213.0
7954,Los Lagos,10,San Juan de la Costa,10306.0,7639.0,2020-12-04,23.0,437.0,10.0,103.0,Osorno,5720.644063,213.0


In [13]:
source=merge2
base=alt.Chart(source)
comparado_act=base.mark_line().transform_aggregate(
activos="mean(casos_activos)", groupby=["region", "fecha"]).encode(
    x=alt.X('fecha:O', axis=alt.Axis(title="Fecha")),
    y=alt.Y('activos:Q', axis=alt.Axis(title='Casos Activos')),
    color=alt.Color('region:N', scale=alt.Scale(scheme="dark2"), legend=alt.Legend(title="Regiones")),
    tooltip=[alt.Tooltip("casos_activos",title="Casos Activos"), alt.Tooltip('region',title="Region"), 
    alt.Tooltip('fecha',title="Fecha")],
    size=alt.Size('poblacion:Q', title='Poblacion'),
).properties(title="Casos Activos vs Tiempo",
    width=400, 
    height=400).interactive()

comparado_conf=base.mark_line().transform_aggregate(
casos_confirmados="max(casos_confirmados)", groupby=["region", "fecha"]).encode(
    x=alt.X('fecha:O', axis=alt.Axis(title="Fecha")),
    y=alt.Y('casos_confirmados:Q', axis=alt.Axis(title='Máximos Confirmados')),
    color=alt.Color('region:N', scale=alt.Scale(scheme="dark2"), legend=alt.Legend(title="Regiones")),
    tooltip=[alt.Tooltip("casos_confirmados",title="Máximos Confirmados"), alt.Tooltip('region',title="Region"), 
    alt.Tooltip('fecha',title="Fecha")],
    size=alt.Size('poblacion:Q', title='Poblacion')
).properties(title="Máximos Confirmados vs Tiempo",
    width=400, 
    height=400).interactive()

comparado_fall=base.mark_line().transform_aggregate(
fallecidos="max(fallecidos)", groupby=["region", "fecha"]).encode(
    x=alt.X('fecha:O', axis=alt.Axis(title="Fecha")),
    y=alt.Y('fallecidos:Q', axis=alt.Axis(title='Máximos Fallecidos')),
    color=alt.Color('region:N', scale=alt.Scale(scheme="dark2"), legend=alt.Legend(title="Regiones")),
    tooltip=[alt.Tooltip("fallecidos",title="Máximos Fallecidos"), alt.Tooltip('region',title="Region"), 
    alt.Tooltip('fecha',title="Fecha")],
    size=alt.Size('poblacion:Q', title='Poblacion')
).properties(title="Máximos Fallecidos s vs Tiempo",
    width=400, 
    height=400).interactive()


alt.vconcat(*(comparado_conf, comparado_fall,comparado_act))

alt.VConcatChart(...)